## Preprocessing

In [ ]:
# Import our dependencies
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import sklearn as skl
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
import numpy as np
from google.colab import files

#  Import and read the charity_data.csv.
import pandas as pd 



In [ ]:
uploaded= files.upload()

Saving charity_data.csv to charity_data (2).csv


In [ ]:
application = pd.read_csv("charity_data.csv")
application.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application = application.drop(["EIN", "NAME"], axis='columns')
application.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:

# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
# application.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
# application['APPLICATION_TYPE'].value_counts()

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
application_counts= application['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(application_counts[application_counts < 500].index)
# application_types_to_replace
# # # # Replace in dataframe
for app in application_types_to_replace:
    application['APPLICATION_TYPE'] = application['APPLICATION_TYPE'].replace(app,"Other")

# # # # Check to make sure binning was successful
application['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
# list(application['CLASSIFICATION'].value_counts())

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
application['CLASSIFICATION'].value_counts()> 1


C1000     True
C2000     True
C1200     True
C3000     True
C2100     True
         ...  
C4120    False
C8210    False
C2561    False
C4500    False
C2150    False
Name: CLASSIFICATION, Length: 71, dtype: bool

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications= application['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classifications[classifications< 1000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application['CLASSIFICATION'] = application['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application.head()

### Columns to PDdummies: APPLICATION_TYPE	AFFILIATION CLASSIFICATION USE_CASE ORGANIZATION INCOME_AMT SPECIAL_CONSIDERATIONS

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
###Create X and Y

In [ ]:
X= application.drop('IS_SUCCESSFUL', axis=1)
X.head()
# Columns to PDdummies: APPLICATION_TYPE	AFFILIATION CLASSIFICATION USE_CASE ORGANIZATION INCOME_AMT SPECIAL_CONSIDERATIONS
X= pd.get_dummies(X, columns=['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS'])
len(X.columns)

43

In [ ]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application['IS_SUCCESSFUL'].values
X = X.values
# Split the preprocessed data into a training and testing dataset
# #  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10) 


In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 8)                 352       
                                                                 
 dense_11 (Dense)            (None, 5)                 45        
                                                                 
 dense_12 (Dense)            (None, 1)                 6         
                                                                 
Total params: 403
Trainable params: 403
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [ ]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled,y_train,epochs=100) 

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6201 - accuracy: 0.6881
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5665 - accuracy: 0.7254
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5605 - accuracy: 0.7283
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5579 - accuracy: 0.7281
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5560 - accuracy: 0.7314
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5548 - accuracy: 0.7311
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7313
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5533 - accuracy: 0.7306
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5525 - accuracy: 0.7320
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5521 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5557 - accuracy: 0.7244 - 321ms/epoch - 1ms/step
Loss: 0.5557171106338501, Accuracy: 0.7244315147399902


In [ ]:
#### Run model adding a hidden layer
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

##Third Hidden Layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 8)                 352       
                                                                 
 dense_14 (Dense)            (None, 5)                 45        
                                                                 
 dense_15 (Dense)            (None, 5)                 30        
                                                                 
 dense_16 (Dense)            (None, 1)                 6         
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
## compile and Fit model with additional layer
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 
fit_model = nn.fit(X_train_scaled,y_train,epochs=100) 

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6526 - accuracy: 0.6437
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5922 - accuracy: 0.7154
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5691 - accuracy: 0.7260
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5611 - accuracy: 0.7278
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5571 - accuracy: 0.7288
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5552 - accuracy: 0.7292
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7292
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5534 - accuracy: 0.7306
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5530 - accuracy: 0.7302
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5524 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5555 - accuracy: 0.7248 - 441ms/epoch - 2ms/step
Loss: 0.5555091500282288, Accuracy: 0.724781334400177


In [ ]:
# Evaluate the model using the sigmoid model for the layers

#### Run model adding a hidden layer
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
hidden_nodes_layer3 = 5
nnsig = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nnsig.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid")
)
# Second hidden layer
#  YOUR CODE GOES HERE
nnsig.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

##Third Hidden Layer
nnsig.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
#  YOUR CODE GOES HERE
nnsig.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nnsig.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 8)                 352       
                                                                 
 dense_18 (Dense)            (None, 5)                 45        
                                                                 
 dense_19 (Dense)            (None, 5)                 30        
                                                                 
 dense_20 (Dense)            (None, 1)                 6         
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
nnsig.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 
fit_model = nnsig.fit(X_train_scaled,y_train,epochs=100) 

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6700 - accuracy: 0.5998
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5861 - accuracy: 0.7276
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5746 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5734 - accuracy: 0.7312
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5730 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5727 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5724 - accuracy: 0.7310
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5721 - accuracy: 0.7307
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5719 - accuracy: 0.7320
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5717 - accura

In [ ]:
### Evaluate SIGModel with 3 layers
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5555 - accuracy: 0.7248 - 309ms/epoch - 1ms/step
Loss: 0.5555091500282288, Accuracy: 0.724781334400177


In [ ]:
##Run the model a second time and drop the SpecialConsiderations & Affiliation Column

# Run Model a second time dropping the Special Considerations tab to see if that changes the outcomes of the model

In [ ]:
app2 = pd.read_csv("charity_data.csv")
app2.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
### Dropping EIN, Name, Special Considerations, AFFILIATION	
app2 = app2.drop(["EIN", "NAME", "SPECIAL_CONSIDERATIONS", "AFFILIATION"], axis='columns')
app2.head()

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,Association,1,0,5000,1
1,T3,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,C3000,ProductDev,Association,1,0,5000,0
3,T3,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
app2_counts= app2['APPLICATION_TYPE'].value_counts()
app2_types_to_replace = list(app2_counts[application_counts < 500].index)
# application_types_to_replace
# # # # Replace in dataframe
for app in app2_types_to_replace:
    app2['APPLICATION_TYPE'] = application['APPLICATION_TYPE'].replace(app,"Other")

# # # # Check to make sure binning was successful
app2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE
app2['CLASSIFICATION'].value_counts()> 1

C1000     True
C2000     True
C1200     True
C3000     True
C2100     True
         ...  
C4120    False
C8210    False
C2561    False
C4500    False
C2150    False
Name: CLASSIFICATION, Length: 71, dtype: bool

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications= app2['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classifications[classifications< 1000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    app2['CLASSIFICATION'] = app2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
app2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [ ]:
#### Convert the categroical --> numerical with PD dummies


X= app2.drop('IS_SUCCESSFUL', axis=1)
X.head()
# Columns to PDdummies: APPLICATION_TYPE	AFFILIATION CLASSIFICATION USE_CASE ORGANIZATION INCOME_AMT SPECIAL_CONSIDERATIONS
X= pd.get_dummies(X, columns=['APPLICATION_TYPE','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT'])
len(X.columns)

35

In [ ]:
### Display X columns to verify successful

X.columns

Index(['STATUS', 'ASK_AMT', 'APPLICATION_TYPE_Other', 'APPLICATION_TYPE_T10',
       'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
       'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T7',
       'APPLICATION_TYPE_T8', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M'],
      dtype='object')

In [ ]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = app2['IS_SUCCESSFUL'].values
X = X.values
# Split the preprocessed data into a training and testing dataset
# #  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10) 

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5
nn2 = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
#  YOUR CODE GOES HERE
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
#  YOUR CODE GOES HERE
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 8)                 288       
                                                                 
 dense_8 (Dense)             (None, 5)                 45        
                                                                 
 dense_9 (Dense)             (None, 1)                 6         
                                                                 
Total params: 339
Trainable params: 339
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
#  YOUR CODE GOES HERE
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [ ]:
# Train the model
#  YOUR CODE GOES HERE
fit_model2 = nn2.fit(X_train_scaled,y_train,epochs=100) 

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6709 - accuracy: 0.6074
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6383 - accuracy: 0.6461
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6281 - accuracy: 0.6518
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6234 - accuracy: 0.6526
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6208 - accuracy: 0.6518
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6192 - accuracy: 0.6526
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6181 - accuracy: 0.6531
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6170 - accuracy: 0.6541
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6161 - accuracy: 0.6544
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6152 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6144 - accuracy: 0.6491 - 404ms/epoch - 2ms/step
Loss: 0.614359438419342, Accuracy: 0.6490961909294128


In [ ]:
### Results from APP2
### Comparison from the first model with dropping the Affiliation and Special Consideration columns decreasing the parameters from 352 to 288.  In which the parameters 
### were decreased the accurace of the model decreased as well.  

In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save('AlphabetSoupCharity_Optimization.h5')
